<a href="https://colab.research.google.com/github/rsotop/BPMN_GPT/blob/main/Redise%C3%B1o_BPMN_GPT_Compartido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Rediseño de procesos de negocio usando BPMN y ChatGPT

Este experimento consiste en generar una propuesta de rediseño de un proceso (to-be) usando ChatGPT, a partir de un archivo XML (.bpmn) que representa la situación actual de un proceso (as-is) generado por la herramienta [BPMN.iO]("https://demo.bpmn.io//new//") de Camunda.


### Configuración de GPT
#### Cargue la clave API y las bibliotecas de Python relevantes.

In [1]:
!pip install openai

import openai
import os
openai.api_key = 'sk-...'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.8 MB/s eta 0:00:00


#### Función auxiliar 
Utilizaremos el modelo gpt-3.5-turbo de OpenAI y el punto final de [Consulte guía](https://platform.openai.com/docs/guides/chat). 

Esta función auxiliar facilitará el uso de indicaciones y la visualización de los resultados generados:

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # este es el grado de aleatoriedad de la salida del modelo
    )
    return response.choices[0].message["content"]

#### Táctica: use delimitadores para indicar claramente distintas partes de la entrada
- Los delimitadores pueden ser cualquier cosa como: ```, """, < >, `<tag> </tag>`, `:`

### Configuración de entorno BPMN.io

Esta función permite visualizar el diagrama BPMN resultante en formato HTML

In [3]:
from IPython.display import HTML

def generate_bpmn_html(bpmn_xml):
    # Código HTML para cargar bpmn-js y mostrar el diagrama
    html_content = f"""
    <html>
        <head>
            <title>BPMN Viewer</title>
            <script src="https://cdn.jsdelivr.net/npm/bpmn-js/dist/bpmn-viewer.production.min.js"></script>
            <style>
                #canvas {{
                    height: 600px;
                    width: 100%;
                }}
            </style>
        </head>
        <body>
            <div id="canvas"></div>
            <script>
                const bpmnXML = `{bpmn_xml}`;
                const viewer = new BpmnJS({{ container: '#canvas' }});

                viewer.importXML(bpmnXML, function(err) {{
                    if (err) {{
                        console.error('Error rendering BPMN diagram', err);
                    }} else {{
                        console.log('BPMN diagram rendered.');
                    }}
                }});
            </script>
        </body>
    </html>
    """

    return html_content

#### Objeto XML (.bpmn) generado en [BPMN.iO]("https://demo.bpmn.io//new//").

In [6]:
bpmnXmlAsIs = f"""
<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" xmlns:di="http://www.omg.org/spec/DD/20100524/DI" id="Definitions_0e75zcs" targetNamespace="http://bpmn.io/schema/bpmn" exporter="bpmn-js (https://demo.bpmn.io)" exporterVersion="12.0.0">
  <bpmn:process id="Process_0hg2idu" isExecutable="false">
    <bpmn:startEvent id="StartEvent_05yz5xt">
      <bpmn:outgoing>Flow_0t51g4g</bpmn:outgoing>
    </bpmn:startEvent>
    <bpmn:task id="Activity_1dcam6f" name="Registrar">
      <bpmn:incoming>Flow_0t51g4g</bpmn:incoming>
      <bpmn:outgoing>Flow_1cq0yap</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_0t51g4g" sourceRef="StartEvent_05yz5xt" targetRef="Activity_1dcam6f" />
    <bpmn:task id="Activity_083ly9r" name="Analizar defecto">
      <bpmn:incoming>Flow_1cq0yap</bpmn:incoming>
      <bpmn:outgoing>Flow_1uwzowl</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_1cq0yap" sourceRef="Activity_1dcam6f" targetRef="Activity_083ly9r" />
    <bpmn:task id="Activity_1qxfuyi" name="Reparar (Compleja)">
      <bpmn:incoming>Flow_1uwzowl</bpmn:incoming>
      <bpmn:outgoing>Flow_0gl6qbs</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_1uwzowl" sourceRef="Activity_083ly9r" targetRef="Activity_1qxfuyi" />
    <bpmn:exclusiveGateway id="Gateway_1v7g0s4">
      <bpmn:incoming>Flow_0gl6qbs</bpmn:incoming>
      <bpmn:outgoing>Flow_02cc8b2</bpmn:outgoing>
      <bpmn:outgoing>Flow_1990s0p</bpmn:outgoing>
      <bpmn:outgoing>Flow_08m238a</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:sequenceFlow id="Flow_0gl6qbs" sourceRef="Activity_1qxfuyi" targetRef="Gateway_1v7g0s4" />
    <bpmn:task id="Activity_192grsq" name="Informar a usuario">
      <bpmn:incoming>Flow_02cc8b2</bpmn:incoming>
      <bpmn:outgoing>Flow_08lguql</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_02cc8b2" sourceRef="Gateway_1v7g0s4" targetRef="Activity_192grsq" />
    <bpmn:task id="Activity_01h1d8v" name="Probar reparación">
      <bpmn:incoming>Flow_1990s0p</bpmn:incoming>
      <bpmn:outgoing>Flow_1s8b3hl</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_1990s0p" sourceRef="Gateway_1v7g0s4" targetRef="Activity_01h1d8v" />
    <bpmn:task id="Activity_1wi5hg7" name="Reparar (simple)">
      <bpmn:incoming>Flow_08m238a</bpmn:incoming>
      <bpmn:outgoing>Flow_1q0od66</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_08m238a" sourceRef="Gateway_1v7g0s4" targetRef="Activity_1wi5hg7" />
    <bpmn:task id="Activity_017ggai" name="Reiniciar reparación">
      <bpmn:incoming>Flow_1q0od66</bpmn:incoming>
      <bpmn:outgoing>Flow_1r7dx1w</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_1q0od66" sourceRef="Activity_1wi5hg7" targetRef="Activity_017ggai" />
    <bpmn:exclusiveGateway id="Gateway_0dsfhde">
      <bpmn:incoming>Flow_1r7dx1w</bpmn:incoming>
      <bpmn:incoming>Flow_08lguql</bpmn:incoming>
      <bpmn:incoming>Flow_1s8b3hl</bpmn:incoming>
      <bpmn:outgoing>Flow_16s0nqt</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:sequenceFlow id="Flow_1r7dx1w" sourceRef="Activity_017ggai" targetRef="Gateway_0dsfhde" />
    <bpmn:sequenceFlow id="Flow_08lguql" sourceRef="Activity_192grsq" targetRef="Gateway_0dsfhde" />
    <bpmn:sequenceFlow id="Flow_1s8b3hl" sourceRef="Activity_01h1d8v" targetRef="Gateway_0dsfhde" />
    <bpmn:task id="Activity_096egnc" name="Archivar reparación">
      <bpmn:incoming>Flow_16s0nqt</bpmn:incoming>
      <bpmn:outgoing>Flow_05uz9lz</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_16s0nqt" sourceRef="Gateway_0dsfhde" targetRef="Activity_096egnc" />
    <bpmn:endEvent id="Event_0tyccj5">
      <bpmn:incoming>Flow_05uz9lz</bpmn:incoming>
    </bpmn:endEvent>
    <bpmn:sequenceFlow id="Flow_05uz9lz" sourceRef="Activity_096egnc" targetRef="Event_0tyccj5" />
  </bpmn:process>
  <bpmndi:BPMNDiagram id="BPMNDiagram_1">
    <bpmndi:BPMNPlane id="BPMNPlane_1" bpmnElement="Process_0hg2idu">
      <bpmndi:BPMNShape id="Activity_1dcam6f_di" bpmnElement="Activity_1dcam6f">
        <dc:Bounds x="310" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_083ly9r_di" bpmnElement="Activity_083ly9r">
        <dc:Bounds x="480" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_1qxfuyi_di" bpmnElement="Activity_1qxfuyi">
        <dc:Bounds x="650" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="_BPMNShape_StartEvent_2" bpmnElement="StartEvent_05yz5xt">
        <dc:Bounds x="192" y="292" width="36" height="36" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Gateway_1v7g0s4_di" bpmnElement="Gateway_1v7g0s4" isMarkerVisible="true">
        <dc:Bounds x="825" y="285" width="50" height="50" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_1wi5hg7_di" bpmnElement="Activity_1wi5hg7">
        <dc:Bounds x="920" y="430" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_017ggai_di" bpmnElement="Activity_017ggai">
        <dc:Bounds x="1090" y="430" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_01h1d8v_di" bpmnElement="Activity_01h1d8v">
        <dc:Bounds x="1000" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_192grsq_di" bpmnElement="Activity_192grsq">
        <dc:Bounds x="1000" y="100" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Gateway_0dsfhde_di" bpmnElement="Gateway_0dsfhde" isMarkerVisible="true">
        <dc:Bounds x="1235" y="285" width="50" height="50" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_096egnc_di" bpmnElement="Activity_096egnc">
        <dc:Bounds x="1350" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Event_0tyccj5_di" bpmnElement="Event_0tyccj5">
        <dc:Bounds x="1522" y="292" width="36" height="36" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNEdge id="Flow_0t51g4g_di" bpmnElement="Flow_0t51g4g">
        <di:waypoint x="228" y="310" />
        <di:waypoint x="310" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1cq0yap_di" bpmnElement="Flow_1cq0yap">
        <di:waypoint x="410" y="310" />
        <di:waypoint x="480" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1uwzowl_di" bpmnElement="Flow_1uwzowl">
        <di:waypoint x="580" y="310" />
        <di:waypoint x="650" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_0gl6qbs_di" bpmnElement="Flow_0gl6qbs">
        <di:waypoint x="750" y="310" />
        <di:waypoint x="825" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_02cc8b2_di" bpmnElement="Flow_02cc8b2">
        <di:waypoint x="850" y="285" />
        <di:waypoint x="850" y="140" />
        <di:waypoint x="1000" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1990s0p_di" bpmnElement="Flow_1990s0p">
        <di:waypoint x="875" y="310" />
        <di:waypoint x="1000" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_08m238a_di" bpmnElement="Flow_08m238a">
        <di:waypoint x="850" y="335" />
        <di:waypoint x="850" y="470" />
        <di:waypoint x="920" y="470" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1q0od66_di" bpmnElement="Flow_1q0od66">
        <di:waypoint x="1020" y="470" />
        <di:waypoint x="1090" y="470" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1r7dx1w_di" bpmnElement="Flow_1r7dx1w">
        <di:waypoint x="1190" y="470" />
        <di:waypoint x="1260" y="470" />
        <di:waypoint x="1260" y="335" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_08lguql_di" bpmnElement="Flow_08lguql">
        <di:waypoint x="1100" y="140" />
        <di:waypoint x="1260" y="140" />
        <di:waypoint x="1260" y="285" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1s8b3hl_di" bpmnElement="Flow_1s8b3hl">
        <di:waypoint x="1100" y="310" />
        <di:waypoint x="1235" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_16s0nqt_di" bpmnElement="Flow_16s0nqt">
        <di:waypoint x="1285" y="310" />
        <di:waypoint x="1350" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_05uz9lz_di" bpmnElement="Flow_05uz9lz">
        <di:waypoint x="1450" y="310" />
        <di:waypoint x="1522" y="310" />
      </bpmndi:BPMNEdge>
    </bpmndi:BPMNPlane>
  </bpmndi:BPMNDiagram>
</bpmn:definitions>
"""

In [7]:
html = generate_bpmn_html(bpmnXmlAsIs)
HTML(html)

In [18]:
prompt = f"""
A partir del objeto XML (.bpmn) proporcionado, que representa la \
situación actual de un proceso de negocio (as-is), genera una \
propuesta de rediseño (to-be). Puede reestructurar actividades, \
introducir o eliminar actividades, proponer otros recursos como la \
automatización. Puede comparar este proceso con otros procesos \
para dar sugerencias sobre cómo mejorarlo. Sea creativo, sin embargo, \
construya su juicio basado en hechos tanto como sea posible.

\"\"\"{bpmnXmlAsIs}\"\"\"
"""
response = get_completion(prompt)
print(response)

Propuesta de rediseño (to-be):

Después de analizar el proceso actual, se propone la siguiente mejora:

1. Automatización de la tarea "Analizar defecto": Se puede utilizar un software de análisis de defectos para automatizar esta tarea y reducir el tiempo de procesamiento.

2. Eliminación de la tarea "Reparar (Compleja)": Esta tarea puede ser eliminada ya que la tarea "Reparar (simple)" puede manejar todos los tipos de reparaciones.

3. Introducción de la tarea "Verificar disponibilidad de piezas": Antes de comenzar la reparación, se debe verificar la disponibilidad de las piezas necesarias para la reparación. Esta tarea puede ser realizada por el personal de compras.

4. Introducción de la tarea "Solicitar piezas faltantes": Si las piezas necesarias para la reparación no están disponibles, se debe solicitarlas al proveedor. Esta tarea puede ser realizada por el personal de compras.

5. Introducción de la tarea "Actualizar registro de reparación": Después de completar la reparación, se

In [19]:
prompt2 = f"""
A partir de la propuesta de rediseño (to-be), generar un nuevo objeto \
XML (.bpmn) que sea interpretable para la herramienta Camunda Modeler.

\"\"\"{response}\"\"\"
"""
bpmnXmlToBe = get_completion(prompt2)
print(bpmnXmlToBe) 

<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL"
xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI"
xmlns:dc="http://www.omg.org/spec/DD/20100524/DC"
xmlns:di="http://www.omg.org/spec/DD/20100524/DI"
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
id="Definitions_1"
targetNamespace="http://bpmn.io/schema/bpmn"
xsi:schemaLocation="http://www.omg.org/spec/BPMN/20100524/MODEL BPMN20.xsd">

<bpmn:process id="Process_1" isExecutable="true">
<bpmn:startEvent id="StartEvent_1"/>
<bpmn:task id="Task_1" name="Analizar defecto">
<bpmn:incoming>SequenceFlow_1</bpmn:incoming>
<bpmn:outgoing>SequenceFlow_2</bpmn:outgoing>
</bpmn:task>
<bpmn:task id="Task_2" name="Automatizar análisis de defectos">
<bpmn:incoming>SequenceFlow_2</bpmn:incoming>
<bpmn:outgoing>SequenceFlow_3</bpmn:outgoing>
</bpmn:task>
<bpmn:task id="Task_3" name="Reparar (simple)">
<bpmn:incoming>SequenceFlow_3</bpmn:incoming>
<bpmn:outgoing>SequenceFlow_4</

In [20]:
html = generate_bpmn_html(bpmnXmlToBe)
HTML(html)

#### Notas sobre el uso de la API de OpenAI

Para instalar la biblioteca OpenAI Python:
```
!pip install openai
```

La biblioteca debe configurarse con la clave secreta de su cuenta, que está disponible en el [sitio web](https://platform.openai.com/account/api-keys). 

Puede configurarlo como la variable de entorno  `OPENAI_API_KEY` antes de usar la biblioteca:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

O bien, establezca `openai.api_key` en su valor:

```
import openai
openai.api_key = "sk-..."

#### Una nota sobre la barra invertida (backslash)
- Se usa una barra invertida \ para hacer que el texto quepa en la pantalla sin insertar caracteres de nueva línea '\n'.
- GPT-3 no se ve realmente afectado si inserta caracteres de nueva línea o no. Pero cuando trabaje con LLM en general, puede considerar si los caracteres de nueva línea en su solicitud pueden afectar el rendimiento del modelo.

### Referencias:
- [GPT-4: A Paradigm Shift in Business Process Improvement](https://www.linkedin.com/pulse/gpt-4-paradigm-shift-business-process-improvement-ihar-suvorau?trackingId=tbLExbPqSHuBUhElhfMivA%3D%3D/?trackingId=tbLExbPqSHuBUhElhfMivA==)
- [ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines)
- [Camunda Academy](https://academy.camunda.com/)
- [BPMN.iO](https://bpmn.io/)
